In [19]:
from graphutils import draw_graph_graphical_props, flat_graph_formulation, Node, VAR, COMP, SOLVER, merge_edges, sources
from graphutils import nested_sources, all_variables, edges_to_Ein_Eout
from nestedgraph import build_typedgraph
from compute import Var
from api import Model, adda, addf, addsolver, setsolvefor
from execution import edges_from_components, generate_components_and_residuals, Component
from operators import sort_scc, reorder_merge_solve
from nestedgraph import build_typedgraph
from workflow import get_f, generate_workflow, implicit_comp_name, SOLVE, OPT, EQ, NEQ, OBJ, IMPL, EXPL, order_from_tree, ENDCOMP, default_solver_options
from workflow import order_from_tree, default_solver_options, mdao_workflow, namefromid
from workflow_mdao import mdao_workflow_with_args
from assembly import architecture_mappings, buildidpvars

In [2]:
import openmdao.api as om

In [3]:
import autograd.numpy as anp

In [4]:
from sympy import exp

In [5]:
z1,z2,x,y2 = Var('z1'), Var('z2'), Var('x'), Var('y2')

In [6]:
model = Model()
m = model.root
a = adda(m, 'a', z2+x-0.2*y2)
y1 = adda(m, 'y1', z1**2+a)
adda(m, y2, y1**0.5+z1+z2)
addf(m, x**2+z2+y1+exp(-y2)) #addobj
addf(m, 3.16-y1) #addineq
addf(m, y2-24) #addineq
setsolvefor(m, [x,z1,z2])

In [7]:
edges = edges_from_components(model.components)
tree = model.Ftree, model.Stree, model.Vtree
#solvers_options = model.solver_options
#comp_options = model.comp_options

In [8]:
nodetyperepr = {VAR: '{}', COMP: 'f_{}', SOLVER: 's_{}'}

In [9]:
G = flat_graph_formulation(*edges)
merge_order = sort_scc(G)

In [10]:
merge_order

[{f_0, f_1, f_2}, {f_3}, {f_5}, {f_4}]

In [11]:
nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)

In [12]:
sequence = order_from_tree(ntree[0], ntree[1], nedges[1])

In [13]:
sequence

[(SOLVER, 1, None),
 (SOLVER, 2, 1),
 (ENDCOMP, [0, 1, 2], 2),
 (ENDCOMP, [3, 5, 4], 1)]

In [14]:
solvers_options = default_solver_options(ntree, {1: {'type': OPT}})

In [15]:
var_options = {'x': [0,10], 'z1': [0,10], 'z2': [0,10]}

In [20]:
wf = mdao_workflow(sequence, solvers_options, {3:OBJ, 4:NEQ, 5:NEQ}, {'x': [0,10], 'z1': [0,10], 'z2': [0,10]})

In [22]:
components = generate_components_and_residuals(model, nedges)

In [23]:
lookup_f = get_f(components, nedges)

In [24]:
namingfunc = namefromid({VAR: '{}', COMP: 'f{}', SOLVER: 's{}'})

In [25]:
wfmdao = mdao_workflow_with_args(wf, lookup_f, namingfunc)

In [26]:
Ftree, Stree, Vtree = ntree
Ein, Eout, Rin = nedges
Ein = merge_edges(Ein, Rin)
# Build MDO model
prob = om.Problem()
mdo_model = prob.model
groups = {None:mdo_model, 'prob':prob}
mdao_in = nested_sources(nedges, ntree, 1)
buildidpvars(mdao_in, mdo_model)

In [27]:
mdao_in

{'x', 'z1', 'z2'}

In [28]:
architecture_mappings[OPT](groups, *wfmdao[0][1:])

In [29]:
architecture_mappings[SOLVE](groups, *wfmdao[1][1:])

In [30]:
architecture_mappings[IMPL](groups, *wfmdao[2][1:])

In [31]:
architecture_mappings[OBJ](groups, *wfmdao[3])

In [32]:
architecture_mappings[NEQ](groups, *wfmdao[4])

In [33]:
architecture_mappings[NEQ](groups, *wfmdao[5])

In [34]:
prob.setup()

In [35]:
prob.set_val('x', 1.0)
prob.set_val('z1', 1.0)
prob.set_val('z2', 1.0)

In [36]:
#prob.run_model()

In [37]:
prob.run_driver();


=====
s1.s2
=====
NL: Newton Converged in 3 iterations

=====
s1.s2
=====
NL: Newton Converged in 0 iterations

=====
s1.s2
=====
NL: Newton Converged in 3 iterations

=====
s1.s2
=====
NL: Newton Converged in 3 iterations

=====
s1.s2
=====
NL: Newton Converged in 2 iterations

=====
s1.s2
=====
NL: Newton Converged in 2 iterations

=====
s1.s2
=====
NL: Newton Converged in 2 iterations
Optimization terminated successfully    (Exit mode 0)
            Current function value: [3.18339414]
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
Optimization Complete
-----------------------------------


In [38]:
allvars = all_variables(*edges_to_Ein_Eout(edges))

In [39]:
xopt = {key: prob.get_val(namingfunc(key, VAR))[0] for key in allvars}

In [42]:
xopt

{'y2': -0.15853055396246019,
 'a': -4.840311002552851,
 'y1': 3.1600000194450577,
 'z2': -4.764651548547751,
 'z1': 2.8284821056527667,
 'x': -0.10736556479759275}

In [41]:
[comp.evaldict(xopt) for comp in components[0:6]]

[[-4.840311002552852],
 [3.160000019445058],
 [-0.15853055396251658],
 -0.4213364381619351,
 -1.9445057564126955e-08,
 -24.15853055396246]

In [39]:
om.n2(prob)

In [40]:
Vtree

{'x': 1, 'z1': 1, 'z2': 1, 'y2': 2, 'a': 2, 'y1': 2}

In [30]:
workflow = generate_workflow(lookup_f, nedges, ntree, solvers_options={1: {'type': OPT}}, comp_options={2:OBJ, 3:NEQ, 4:NEQ})

In [31]:
workflow

[(OPT, None, 's_1', {'type': OPT, 'designvars': ('x', 'z1', 'z2')}),
 (SOLVE, 's_1', 's_2', {'type': SOLVE}),
 (IMPL,
  's_2',
  'res_f0_f1',
  [(['x_y2', 'x_z1', 'x_z2', 'x_x', 'x_y1'],
    ['x_y1'],
    <bound method Component.evaldict of (('y2', 'z1', 'z2', 'x', 'y1'), 0, (None,), 'None')>,
    <bound method Component.graddict of (('y2', 'z1', 'z2', 'x', 'y1'), 0, (None,), 'None')>,
    1.0),
   (['x_y1', 'x_z1', 'x_z2', 'x_y2'],
    ['x_y2'],
    <bound method Component.evaldict of (('y1', 'z1', 'z2', 'y2'), 1, (None,), 'None')>,
    <bound method Component.graddict of (('y1', 'z1', 'z2', 'y2'), 1, (None,), 'None')>,
    1.0)]),
 (OBJ,
  s_1,
  OBJ,
  (('y1', 'y2', 'z1', 'x'), 2, (None,), 'x**2 + y1 + z1 + exp(-y2)')),
 (NEQ, s_1, NEQ, (('y2',), 4, (None,), 'y2 - 24')),
 (NEQ, s_1, NEQ, (('y1',), 3, (None,), '3.16 - y1'))]